We need a unitary $U$ such $U: input \to output$. 

$input = \ket{\psi} \otimes (\frac{1}{\sqrt{2}}(\ket{00} + \ket{11}))^{\otimes oq} $

$oq$ stands for operation qubits(is the number of epr pairs).

$output = \sum_{k} \sqrt{\alpha_{k}} U_{k} \ket{\psi}\ket{k} $, where $U_{k}$ is the error associated with the output $\ket{k}$ 

Because of the non locality constrain $U$ must be on the form $U = U_{A} \otimes U_{B} $

In [94]:
def basis_vectors(n):
    basis = []
    for i in range(2**n):
        word = ""
        aux = i
        for j in range(n):
            if((aux)%2 == 0):
                word = "0" + word
            else:
                word = "1" + word
            
            aux = aux//2 
            
        basis.append(word)
    return basis

# print(basis_vectors(4))

In [95]:
import numpy as np

def initialize(coef, neprs):
    psi1 = np.array([coef[0], coef[1]])
    psi2 = np.array([coef[2], coef[3]])

    encodingA = psi1
    encodingB = psi2

    # building the eprs pairs by verifying the basis states that corresponds to a possible state.
    # we are considering that the first half qubits are in A and the other half is in B.
    # the first epr qubit in A makes a pair with the first epr qubit in B.

    n_qubits = 2*neprs
    basis = basis_vectors(n_qubits)
    eprs = np.zeros(len(basis))
    c = 1 / np.sqrt(2**((n_qubits)//2))

    for i in range(len(basis)): 
        flag = 0
        for l in range(len(basis[i])//2):
            if basis[i][l] != basis[i][(len(basis[i])//2)+l]:
                flag = 1
        if flag == 0:
            eprs[i] = c

    # eprsA = [1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]
    # eprsB = [1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]

    # for j in range(neprs//2 -1):
    #     eprsA = np.kron(eprsA,eprsA)
    #     eprsB = np.kron(eprsB,eprsB)

    # entire = np.kron(np.kron(np.kron(encodingA, eprsA), encodingB), eprsB)

    
    entire = np.kron(np.kron(encodingA, eprs), encodingB)

    return encodingA, eprs, encodingB, entire

def test():
    neprs = 3
    n_qubits = 2*neprs
    basis = basis_vectors(n_qubits)
    eprs = np.zeros(len(basis))
    c = 1 / np.sqrt(2**((n_qubits)//2))

    for i in range(len(basis)): 
        flag = 0
        for l in range(len(basis[i])//2):
            if basis[i][l] != basis[i][(len(basis[i])//2)+l]:
                flag = 1
        if flag == 0:
            eprs[i] = c

    print(basis)
    print(eprs)
# test()

The ideia is to construct an output using unitary transformations.

$U \times input = output$

We get the number of inputs needed to make a matrix equation, then calculate the inverse of the input and then get $U$

In [96]:
def mapping(basis):
    mapping = {}
    i = 0
    for item in basis:
        mapping[item] =  item[-1] + item[0]
        print(item, " --> ", mapping[item],end="   ;   ")
        if i%4 == 3:
            print()
        i += 1

    return mapping



$U$ is NxN where N is $2^{n}$, where n is the number of qubits.

We need N inputs and outputs to make the matrix equation.

In [97]:
# alpha = np.sqrt(3)/np.sqrt(4)
# beta = 1/np.sqrt(4)
alpha = 1/np.sqrt(2)
beta = 1/np.sqrt(2)
# gamma = 1/np.sqrt(2)
# delta = 1/np.sqrt(2)
# gamma = np.sqrt(2)/np.sqrt(3)
# delta = 1/np.sqrt(3)
gamma = 0
delta = 1

coef = [alpha, beta, gamma, delta]
neprs = 1

encodingA, eprs, encodingB, entire = initialize(coef, neprs)

total_n = int(np.log2(len(coef))) + 2*neprs

print("State of encoding qubits A: ", encodingA)
print("State of encoding qubits B: ", encodingB)
print("State of epr pairs: ", eprs)
print("Basis:",basis_vectors(total_n))
print("State of the entire system: ", entire, len(entire))

State of encoding qubits A:  [0.70710678 0.70710678]
State of encoding qubits B:  [0 1]
State of epr pairs:  [0.70710678 0.         0.         0.70710678]
Basis: ['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']
State of the entire system:  [0.  0.5 0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.5] 16


To fullfill all the inputs that are need we gonna use the computational basis for the encoding qubits and the bell basis for the bell pair.

I have only defined the output for the inputs with the epr pairs, should I put the same output for all inputs with the same $\ket{\psi}$ ?

In [98]:
c = 1/np.sqrt(2)
bell_basis = [[1/np.sqrt(2),0,0,1/np.sqrt(2)],
                       [1/np.sqrt(2),0,0,-1/np.sqrt(2)],
                       [0,1/np.sqrt(2),1/np.sqrt(2),0],
                       [0,1/np.sqrt(2),-1/np.sqrt(2),0]]

def make_bell_inputs():
    bell_inputs = []
    for i in range(len(bell_basis)):
        for j in range(len(bell_basis)):
            bell_inputs.append(np.kron(bell_basis[i], bell_basis[j]))
    return bell_inputs

def correct_order(bell_input):
    # this is correct only for 2 epr pairs, the first on A is the pair of the first on B and so on
    correct = np.zeros(len(bell_input))
    order = [0,1,4,5,2,3,6,7,8,9,12,13,10,11,14,15]
    for i in range(len(correct)):
        correct[i] = bell_input[order[i]]
    return correct

In [99]:
nA = 1
nB = 1
neprs = 2
total_n = nA + nB + 2*neprs
N = 2**total_n
basis = basis_vectors(total_n)

#building N inputs by building A then the eprs then B

bell_inputs = make_bell_inputs()

inputs0 = []
for i in range(2**(nA)):
    inputA = np.zeros(2**(nA))
    inputA[i] = 1
    
    for j in range(2**(2*neprs)):
        inputEpr = correct_order(bell_inputs[j])

        for k in range(2**(nB)):
            inputB = np.zeros(2**(nB))
            inputB[k] = 1
            
            inputs0.append(np.kron(np.kron(inputA,inputEpr),inputB))

print(inputs0)  
 
# print(len(inputs))
# print(len(inputs[0]))


[array([0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]), array([0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]), array([ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,
       -0. ,  0. ,  0. , -0. , -0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
        0. ,  0. ,  0. ,  0. , -0. , -0. ,  0. ,  0. , -0.5, -0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0. ,

In [100]:
#Use one qubit gates based on the purification equation
I = np.matrix([[1,0],[0,1]])
Z = np.matrix([[1,0],[0,-1]])
X = np.matrix([[0,1],[1,0]]) 

I5 = np.kron(np.kron(np.kron(np.kron(I,I),I),I),I)

IIIIIX = np.kron(I5,X)
XIIIII = np.kron(X,I5)
IIIIIZ = np.kron(I5,Z)
ZIIIII = np.kron(Z,I5)



In [101]:
print(inputs0[1])

computational_basis_inputs = []
for i in range(2**(total_n)):
    inputc = np.zeros(2**(total_n))
    inputc[i] = 1
    computational_basis_inputs.append(inputc)

def check(input,i):
    term = computational_basis_inputs[i].copy()
    if input[2] == 1:
        term = IIIIIX @ term
    if input[3] == 1:
        term = XIIIII @ term
    if input[1] == 1:
        term = IIIIIZ @ term
    if input[4] == 1:
        term = ZIIIII @ term
    
    return term

[0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]


In [102]:
out00 = np.zeros(N)
out01 = np.zeros(N)
out10 = np.zeros(N)
out11 = np.zeros(N)

n00 = 0
n01 = 0
n10 = 0
n11 = 0

for i in range(len(computational_basis_inputs)):
    #if the input2[i] has a non 0 value
    term = check(basis[i],i)
    if basis[i][0] == "0" and basis[i][-1] == "0":
        out00 += term
        n00 += 1
    if basis[i][0] == "0" and basis[i][-1] == "1":
        out01 += term
        n00 += 1
    if basis[i][0] == "1" and basis[i][-1] == "0":
        out10 += term
        n00 += 1
    if basis[i][0] == "1" and basis[i][-1] == "1":
        out11 += term
        n11 += 1

out00 = (1/np.sqrt(n00))*out00
out01 = (1/np.sqrt(n00))*out01
out10 = (1/np.sqrt(n00))*out10
out11 = (1/np.sqrt(n00))*out11

In [133]:
# building the output
outputs0 = []

for i in range(N//4):
    outputs0.append(out00)
    outputs0.append(out01)

for i in range(N//4):
    outputs0.append(out10)
    outputs0.append(out11)

print(len(inputs0))
print(len(outputs0))
print(len(inputs0[0]))
print(len(outputs0[0]))

# output = np.linalg.inv(outputs0)
output = np.transpose(outputs0)
input = np.transpose(inputs0)
input_inv = np.linalg.inv(input)

U = output @ input_inv
print("len U ",len(U))
print("len U[0] ",len(U[0]))
# print(U)


def is_unitary(matrix: np.ndarray) -> bool:
    unitary = True
    error = np.linalg.norm(np.eye(N) - matrix.dot( matrix.transpose().conjugate()))

    if not(error < np.finfo(matrix.dtype).eps * 10.0 *N):
        unitary = False

    return unitary

print(is_unitary(U))


64
64
64
64
len U  64
len U[0]  64
False
